In [40]:
import os

import rasterio
from rasterio.plot import reshape_as_image
import rasterio.mask
from rasterio.features import rasterize

import pandas as pd
import geopandas as gpd
from shapely.geometry import mapping, Point, Polygon
from shapely.ops import unary_union

import numpy as np
import cv2
import matplotlib.pyplot as plt

import geopandas as gpd

REF_ROOT = "/Users/jmccarty/Data/220206_ReTo_carto/template_matching/ref"
DATA_ROOT = "/Users/jmccarty/Data/220206_ReTo_carto/template_matching/tif_files"
SHEETS = ["1092_1990","1111_1988","1132_1989"]
QATM_ROOT = "/Users/jmccarty/GitHub/22spring_templatematching_carto/QATM/Phase3_result"
RESULTS_PATH = "/Users/jmccarty/GitHub/22spring_templatematching_carto/QATM/notebooks/evaluate"

In [42]:
def generate_mask(raster_path, shape_path, output_path):
    """
    Function that generates a binary mask from a vector file (shp or geojson)
    raster_path = path to the .tif;
    shape_path = path to the shapefile or GeoJson.
    output_path = Path to save the binary mask.
    file_name = Name of the file.
    """
    
    #load raster
    
    with rasterio.open(raster_path, "r") as src:
        raster_img = src.read()
        raster_meta = src.meta
    
    #load o shapefile ou GeoJson
    train_df = gpd.read_file(shape_path)
    
    #Verify crs
    if train_df.crs != src.crs:
        print(" Raster crs : {}, Vector crs : {}.\n Convert vector and raster to the same CRS.".format(src.crs,train_df.crs))
        
        
    #Function that generates the mask
    def poly_from_utm(polygon, transform):
        poly_pts = []

        poly = unary_union(polygon)
        for i in np.array(poly.exterior.coords):

            poly_pts.append(~transform * tuple(i))

        new_poly = Polygon(poly_pts)
        return new_poly
    
    
    poly_shp = []
    im_size = (src.meta['height'], src.meta['width'])
    for num, row in train_df.iterrows():
        if row['geometry'].geom_type == 'Polygon':
            poly = poly_from_utm(row['geometry'], src.meta['transform'])
            poly_shp.append(poly)
        else:
            for p in row['geometry']:
                poly = poly_from_utm(p, src.meta['transform'])
                poly_shp.append(poly)

    mask = rasterize(shapes=poly_shp,
                     out_shape=im_size)
    
    #Salve
    mask = mask.astype("uint16")
    
    bin_mask_meta = src.meta.copy()
    bin_mask_meta.update({'count': 1})
    
    with rasterio.open(output_path, 'w', **bin_mask_meta) as dst:
        dst.write(mask * 255, 1)

In [ ]:
# quickly fix the original tif files which have no CRS
# this only needed to be run once

for sheet in SHEETS:
    ref_path = os.path.join(DATA_ROOT, f"LKg_{sheet}.tif")
    r = rasterio.open(ref_path, "r+")
    print(r.read_crs())
    r.crs = rasterio.crs.CRS({"init": "epsg:21781"})
    print(r.read_crs())
    r.close()

In [43]:
sheet = SHEETS[0]
ref_shape_path = os.path.join(REF_ROOT,f"LKg_{sheet}",f"wetlands_{sheet}.shp")
ref_shape = gpd.read_file(ref_shape_path)
ref_image_path = os.path.join(DATA_ROOT, f"LKg_{sheet}.tif")
qatm_shape_path = os.path.join(QATM_ROOT,f"LKg_{sheet}_maskshape.geojson")
qatm_shape = gpd.read_file(qatm_shape_path)
output_ref_mask_path = os.path.join(RESULTS_PATH,f"{sheet}_ref_shp_mask.tif")
output_qatm_mask_path = os.path.join(RESULTS_PATH,f"{sheet}_qatm_shp_mask.tif")

In [44]:
# generate mask files for the qatm_shapes
generate_mask(ref_image_path,qatm_shape_path,output_qatm_mask_path)

# generate mask files for the original shapes
generate_mask(ref_image_path,ref_shape_path,output_ref_mask_path)


/var/folders/ww/12z6q0wj4c527ftkk7sxqb680000gp/T/ipykernel_26701/3785567254.py:44: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for p in row['geometry']:
/var/folders/ww/12z6q0wj4c527ftkk7sxqb680000gp/T/ipykernel_26701/3785567254.py:44: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for p in row['geometry']:


In [52]:
# load the masks

ref_mask = rxr.open_rasterio(output_ref_mask_path)
qatm_mask = rxr.open_rasterio(output_qatm_mask_path)

/opt/homebrew/Caskroom/miniforge/base/envs/cartography/lib/python3.8/site-packages/pyproj/crs/_cf1x8.py:511: UserWarning: angle from rectified to skew grid parameter lost in conversion to CF
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/cartography/lib/python3.8/site-packages/pyproj/crs/_cf1x8.py:511: UserWarning: angle from rectified to skew grid parameter lost in conversion to CF
  warnings.warn(


In [74]:
# cellulate the arrays

cell_width = 24
cell_height = 35

total_width = ref_mask[0].shape[0]
total_height = ref_mask[0].shape[1]

width_indices = np.arange(0,total_width,cell_width)
height_indices = np.arange(0,total_height,cell_height)

indices = []
for i in width_indices:
    for j in height_indices:
        indices.append((i,j))

In [82]:
for inx in indices:
    i = inx[0]
    j = inx[1]

    ref_sub = ref_mask[0].values[i:i+cell_width,j:j+cell_height]
    qatm_sub = qatm_mask[0].values[i:i+cell_width,j:j+cell_height]
    arr_comp = ref_sub - qatm_sub
    if sum(arr_comp.flatten())>0:
        print("Yes")

Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
